<a href="https://colab.research.google.com/github/UdayVT/Black-Scholes-Neural-Network/blob/main/Black_Scholes_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from scipy.stats import norm

import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, LeakyReLU, Dropout
from keras import backend
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.utils import get_custom_objects
from keras import backend as K
from keras.activations import elu
import tensorflow as tf

In [2]:
N = norm.cdf
def BS_CALL(S, K, T, r, q , sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S*(math.exp(-q*T))* N(d1) - K * np.exp(-r*T)* N(d2)

In [3]:
price = np.random.uniform(10, 20, 300000)
strike = np.random.uniform(10, 20, 300000)
time = np.random.uniform(0.1, 3, 300000)
intrest = np.random.uniform(0.01, 0.03, 300000)
divident = np.random.uniform(0, 0.03, 300000)
volatility = np.random.uniform(0.05, 0.3, 300000)
call = np.zeros(300000)

In [4]:
for i in range(0,300000):
  call[i] = BS_CALL(price[i],strike[i],time[i],intrest[i],divident[i],volatility[i])

In [5]:
df = pd.DataFrame()

In [6]:
df['Stock'] = price.tolist()
df['Strike'] = strike.tolist()
df['Time'] = time.tolist()
df['Intrest'] = intrest.tolist()
df['Divident'] = divident.tolist()
df['Volatility'] = volatility.tolist()
df['Call'] = call.tolist()

In [7]:
df_filtered = df[df['Call'] <= 10]
df_filtered = df_filtered[df_filtered['Call'] >= 0.001]

In [8]:
df_filtered['Stock'] = df_filtered['Stock'].div(df_filtered.Strike)
df_filtered['Call'] = df_filtered['Call'].div(df_filtered.Strike)
MinCall = df_filtered.loc[df_filtered["Call"].idxmin()].Call
df_filtered['Strike'] = 1
Strike = df_filtered.Strike.values
df_filtered = df_filtered.drop('Strike', axis=1)

In [9]:
df_filtered['Call'] = df_filtered['Call'] - 0.5

In [10]:
prices = df_filtered.values

In [11]:
X = prices[:,:5]
y = prices[:,5:6]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
def custom_activation(x):
    return (K.softplus(x)-0.5)

get_custom_objects().update({'custom_activation': Activation(custom_activation)})

In [13]:
def MELU(z):
    cond = K.greater(z, 0)
    return K.switch(cond, (((0.5*(K.elu(z)**2))+(0.02*K.elu(z)))/(z-0.04081633)),K.elu(z,0.49))

get_custom_objects().update({'MELU': Activation(MELU)})

In [14]:
#def MELU(x,alpha=0.49):
#  cond = K.greater(x, 0)
#  a = 1-(2*alpha)
#  b = -2 + (1/alpha)
#  return K.switch(cond,((0.5 * (x**2)+(a*x))/(x+b)),K.elu(x,alpha=0))

#get_custom_objects().update({'MELU': Activation(MELU)})

In [15]:
#def melu(x, alpha=0.49):
#    a = 1-(2*alpha)
#    b = -2 + (1/alpha)
#    return np.where(x >= 0.0,((0.5 * (x**2)+(a*x))/(x+b)), alpha * (np.exp(x) - 1.0))

#class MELU(keras.layers.Layer):
#    def __init__(self, alpha=0.49):
#        super().__init__()
#        self.alpha = alpha
#
#    def call(self, x):
#        return melu(x, alpha=self.alpha)

In [16]:
from pandas.core.arrays.arrow.array import Any
nodes = 128

# Create Model
model = Sequential()

# Layer 1
model.add(Dense(nodes, input_shape=(5,)))
model.add(Activation(LeakyReLU(1.0)))

# Layer 2
model.add(Dense(nodes))
model.add(Activation (elu))

# Layer 3
model.add(Dense(nodes))
model.add(Activation (elu))

# Layer 4 - Output Layer
model.add(Dense(1))
model.add(Activation(custom_activation))

In [17]:
model.compile(loss='mse',
        optimizer='rmsprop')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               768       
                                                                 
 activation_2 (Activation)   (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 activation_3 (Activation)   (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 activation_4 (Activation)   (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [ ]:
model.fit(X_train, y_train, epochs = 15, batch_size = 64)

Epoch 1/15
3441/3441 [==============================] - 9s 2ms/step - loss: 0.0039
Epoch 2/15
3441/3441 [==============================] - 7s 2ms/step - loss: 6.6197e-04
Epoch 3/15
3441/3441 [==============================] - 8s 2ms/step - loss: 4.5285e-04
Epoch 4/15
3441/3441 [==============================] - 7s 2ms/step - loss: 3.5966e-04
Epoch 5/15
3441/3441 [==============================] - 8s 2ms/step - loss: 3.0187e-04
Epoch 6/15
3441/3441 [==============================] - 7s 2ms/step - loss: 2.5989e-04
Epoch 7/15
3441/3441 [==============================] - 10s 3ms/step - loss: 2.2772e-04
Epoch 8/15
3441/3441 [==============================] - 8s 2ms/step - loss: 2.0548e-04
Epoch 9/15
3441/3441 [==============================] - 7s 2ms/step - loss: 1.8869e-04
Epoch 10/15
3441/3441 [==============================] - 8s 2ms/step - loss: 1.7612e-04
Epoch 11/15
3441/3441 [==============================] - 7s 2ms/step - loss: 1.6384e-04
Epoch 12/15
3441/3441 [=====================

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
#out sample(20)
plt.figure(figsize = (5,10))
plt.scatter((y_test+0.5)*10, (predictions+0.5)*10)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.plot([0,10], [0,10], 'r')
plt.grid(True)

In [ ]:
InSamplePredictions = model.predict(X_train)

In [ ]:
#in sample (80)
plt.figure(figsize = (5,10))
plt.scatter((y_train+0.5)*10, (InSamplePredictions+0.5)*10)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.plot([0,10], [0,10], 'r')
plt.grid(True)